In [19]:
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix
import csv
from geopy.distance import geodesic
import math


def calculate_distance_matrix(df):
    return pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)

def find_nearest_neighbour(df, current_location):
    return df.loc[current_location].idxmin()

def greedy_algorithm(df, depot):
    unvisited = df.columns.tolist()
    unvisited.remove(depot)
    current_location = depot
    route = [depot]

    while unvisited:
        next_location = find_nearest_neighbour(df[unvisited], current_location)
        route.append(next_location)
        unvisited.remove(next_location)
        current_location = next_location

    route.append(depot)  # return to depot
    return route

def calculate_distance(route, data):
    total_distance = 0
    for i in range(len(route) - 1):
        loc1 = (data.loc[route[i], 'lat'], data.loc[route[i], 'lng'])
        loc2 = (data.loc[route[i+1], 'lat'], data.loc[route[i+1], 'lng'])
        total_distance += round(geodesic(loc1, loc2).km)
    return total_distance

def main():
    datasets = {'part_b_input_dataset_1.csv': 20, 'part_b_input_dataset_2.csv': 30}
    for dataset, capacity in datasets.items():
        # Load your data
        data = pd.read_csv(dataset)
        data.set_index('order_id', inplace=True)

        # Calculate distance matrix
        distance_df = calculate_distance_matrix(data)

        # Initialize vehicles
        vehicles = [{'capacity': capacity, 'route': []} for _ in range(2)]  # Use the capacity from the dictionary

        # Assign routes to vehicles
        for vehicle in vehicles:
            while len(vehicle['route']) < vehicle['capacity'] and not distance_df.empty:
                depot = distance_df.index[0]
                route = greedy_algorithm(distance_df, depot)
                vehicle['route'].extend(route[:vehicle['capacity'] + 1])  # +1 to include return to depot
                distance_df.drop(index=route[:vehicle['capacity']], columns=route[:vehicle['capacity']], inplace=True)

        # Define depot latitude and longitude
        depot_lat = 43.871
        depot_lng = 126.5661

        # Write routes to CSV
        with open(f'part_b_output_{dataset}', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["order_id", "lng", "lat", "depot_lat", "depot_lng", "vehicle_num", "dlvr_seq_num"])
            for i, vehicle in enumerate(vehicles, start=1):
                for j, order_id in enumerate(vehicle['route'], start=1):
                    if order_id != depot:
                        writer.writerow([order_id, data.loc[order_id, 'lng'], data.loc[order_id, 'lat'], depot_lat, depot_lng, i, j])

        # Calculate and write route distances to CSV
        with open('part_b_routes_distance_travelled.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            distances = [f'part_b_output_{dataset}']
            total_distance = 0
            for vehicle in vehicles:
                route_distance = calculate_distance(vehicle['route'], data)
                total_distance += route_distance
                distances.append(route_distance)
            distances.append(total_distance)
            writer.writerow(distances)

if __name__ == "__main__":
    main()